# Agent Executor From Scratch

In this notebook we will go over how to build a basic agent executor from scratch.

## Setup¶
First we need to install the packages required

In [1]:
# %%capture --no-stderr
# %pip install --quiet -U langchain langchain_openai langchainhub tavily-python

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [2]:
import os

LOCAL_LLM = "llama3"
os.environ["TAVILY_API_KEY"] = "<>"

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [3]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "<>"

## Create the LangChain agent

First, we will create the LangChain agent. For more information on LangChain agents, see [this documentation](https://python.langchain.com/docs/modules/agents/)

In [4]:
from langchain import hub
from langchain_core.prompts.prompt import PromptTemplate
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/react")

prompt_template = \
"""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following conditional format:

1. If you found the answer, generate the output in the following format:
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

2. But if you don't know the answer yet, generate the output in the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, MUST be one of [{tool_names}], other values are NOT allowed
    Action Input: the input to the action
    Observation: the result of the action

Note that the second condition can repeat N times until you find the answer.
The history of previous actions and their corresponding observations is available in 'intermediate_steps' attribute.
It can help avoid completely repetitive actions.
Feel free to do a repetitive Action with different Action Input if you need more information.

Begin!

Question: {input}

Thought: {agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)

# Choose the LLM that will drive the agent
llm = ChatOllama(model=LOCAL_LLM, temperature=0)

# Construct the OpenAI Functions agent
agent_runnable = create_react_agent(llm, tools, prompt)

In [5]:
prompt.pretty_print()


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following conditional format:

1. If you found the answer, generate the output in the following format:
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

2. But if you don't know the answer yet, generate the output in the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, MUST be one of [{tool_names}], other values are NOT allowed
    Action Input: the input to the action
    Observation: the result of the action

Note that the second condition can repeat N times until you find the answer.
The history of previous actions and their corresponding observations is available in 'intermediate_steps' attribute.
It can help avoid completely repetitive actions.
Feel free to do a repetitive Action with different Action Input if you n

## Define the graph state

We now define the graph state. The state for the traditional LangChain agent has a few attributes:

1. `input`: This is the input string representing the main ask from the user, passed in as input.
2. `chat_history`: This is any previous conversation messages, also passed in as input.
3. `intermediate_steps`: This is list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent.
4. `agent_outcome`: This is the response from the agent, either an AgentAction or AgentFinish. The AgentExecutor should finish when this is an AgentFinish, otherwise it should call the requested tools.


In [6]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Define the nodes

We now need to define a few different nodes in our graph.
In `langgraph`, a node can be either a function or a [runnable](https://python.langchain.com/docs/expression_language/).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken.
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [7]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

## Define the graph

We can now put it all together and define the graph!

In [8]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [9]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='tavily_search_results_json', tool_input='what is the weather in San Francisco', log='Let\'s start by searching for the answer.\n\nQuestion: what is the weather in sf\nThought: I should always think about what to do\nAction: tavily_search_results_json\nAction Input: "what is the weather in San Francisco')}
----
{'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='what is the weather in San Francisco', log='Let\'s start by searching for the answer.\n\nQuestion: what is the weather in sf\nThought: I should always think about what to do\nAction: tavily_search_results_json\nAction Input: "what is the weather in San Francisco'), '[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1715109143, \'localtime\': \'2024-05

In [10]:
inputs = {"input": "بهترین شهرهای ایران برای مسافرت تابستانی چیست؟", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='tavily_search_results_json', tool_input='بهترین شهرهای ایران برای مسافرت تابستانی چیست', log="A question about the best cities in Iran for summer travel!\n\nSince I have access to `tavily_search_results_json`, I'll use it to find the answer.\n\nHere's my first action:\n\nAction: tavily_search_results_json\nAction Input: بهترین شهرهای ایران برای مسافرت تابستانی چیست")}
----
{'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='بهترین شهرهای ایران برای مسافرت تابستانی چیست', log="A question about the best cities in Iran for summer travel!\n\nSince I have access to `tavily_search_results_json`, I'll use it to find the answer.\n\nHere's my first action:\n\nAction: tavily_search_results_json\nAction Input: بهترین شهرهای ایران برای مسافرت تابستانی چیست"), "[{'url': 'https://faragasht.com/the-best-cities-in-iran-to-travel-in-summer/', 'content': 'قشم زیبا را فراموش نکن. یکی دیگر از بهترین شهرهای ایران برای سفر در تابستان، جزیره